In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.circuit.library import TwoLocal


import numpy as np
import pandas as pd
from qiskit_algorithms.utils import algorithm_globals
import networkx as nx
from networkx.algorithms import bipartite

import matplotlib.pyplot as plt

import random

from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Binary, maximize
from scipy.optimize import minimize
from itertools import combinations

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.visualization import *
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.primitives import Estimator
from qiskit.primitives import Sampler,  BackendSampler, BackendSamplerV2

from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.circuit.library import QAOAAnsatz

from qiskit.circuit.library import EfficientSU2

from qiskit_algorithms.optimizers import SPSA

from qiskit_algorithms import SamplingVQE
from qiskit.primitives import Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA


In [3]:
n_donors = 5  # Number of donors
n_recipients = 5  # Number of recipients
weights = [
    [10, 2, 1, 6, 8],  # Donor 0
    [5, 9, 4, 9, 11],   # Donor 1
    [7, 3, 2, 1, 8],  # Donor 2
    [6, 8, 1, 3, 5],  # Donor 3
    [1, 2, 5, 6, 7]    # Donor 4
]  # Example weights

qubo = QuadraticProgram()

for i in range(n_donors):
    qubo.binary_var(name=f"donor_{i}")

for j in range(n_recipients):
    qubo.binary_var(name=f"recipient_{j}")

objective_coeffs = {}
for i in range(n_donors):
    for j in range(n_recipients):
        objective_coeffs[(f"donor_{i}", f"recipient_{j}")] = weights[i][j]

qubo.maximize(quadratic=objective_coeffs)

backend = AerSimulator(max_parallel_shots=1)

# Set up the optimizer
optimizer = COBYLA()
sampler = BackendSampler(backend=backend)

# Set up the QAOA with the Estimator primitive
estimator = Estimator()
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=4) 
qubo_to_qubo = QuadraticProgramToQubo()

qubo_model = qubo_to_qubo.convert(qubo)

minimum_eigen_optimizer = MinimumEigenOptimizer(qaoa)
print("Here!")
result = minimum_eigen_optimizer.solve(qubo)

# Extract results
print("Solution:", result)
print("Objective value:", result.fval)
#result = qaoa.compute_minimum_eigenvalue(operator=ising[0])

Here!
Solution: fval=130.0, donor_0=1.0, donor_1=1.0, donor_2=1.0, donor_3=1.0, donor_4=1.0, recipient_0=1.0, recipient_1=1.0, recipient_2=1.0, recipient_3=1.0, recipient_4=1.0, status=SUCCESS
Objective value: 130.0


In [4]:
# Extract results
print("Solution:", result)
print("Objective value:", result.fval)
#result = qaoa.compute_minimum_eigenvalue(operator=ising[0])

Solution: fval=130.0, donor_0=1.0, donor_1=1.0, donor_2=1.0, donor_3=1.0, donor_4=1.0, recipient_0=1.0, recipient_1=1.0, recipient_2=1.0, recipient_3=1.0, recipient_4=1.0, status=SUCCESS
Objective value: 130.0
